# Laboratorio: Feature Engineering con el dataset Wine Quality

En este notebook entrenarás un modelo de clasificación para predecir si un vino es de
**buena calidad** o no, utilizando el dataset público *Wine Quality* de UCI.

1. Primero ejecutaremos un modelo **sin Feature Engineering** (solo con las variables originales).
2. Después, en la sección marcada como:

```python
# ============================================
# SECCIÓN PARA FEATURE ENGINEERING (A COMPLETAR)
# ============================================
```

deberás **crear nuevas características y/o aplicar transformaciones** (por ejemplo,
transformaciones logarítmicas, relaciones entre variables, escalado, etc.) y volver
a entrenar el modelo para analizar la mejora en el desempeño.

Alumno: Victor Manuel Telles Amezcua | 737066

In [ ]:
# ============================
# 1. IMPORTACIÓN DE LIBRERÍAS
# ============================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt

In [ ]:
# ==================
# 2. CARGA DE DATOS
# ==================

# IMPORTANTE:
# Descarga el archivo winequality-red.csv o winequality-white.csv desde UCI
# y colócalo en la misma carpeta que este notebook.
# En UCI el separador es ';', por eso usamos sep=';'.

csv_path = "data/winequality-red.csv"  # <- Cambia a winequality-white.csv si lo prefieres

data = pd.read_csv(csv_path, sep=';')

print("Primeras filas del dataset:")
display(data.head())

print("\nInformación general del dataset:")
print(data.info())

print("\nEstadísticas descriptivas:")
display(data.describe())

In [ ]:
# ======================================================
# 3. CREACIÓN DE LA VARIABLE OBJETIVO BINARIA (LABEL)
# ======================================================

# La columna 'quality' toma valores enteros (por ejemplo, de 3 a 8).
# Definimos "buen vino" si quality >= 6, y "no tan bueno" si quality <= 5.

data['good_quality'] = (data['quality'] >= 6).astype(int)

# Variable objetivo
y = data['good_quality']

# Variables de entrada: todas las columnas excepto 'quality' y 'good_quality'
X = data.drop(columns=['quality', 'good_quality'])

print("Columnas de entrada (features):")
print(X.columns)

print("\nDistribución de la variable objetivo (0 = calidad baja, 1 = calidad alta):")
print(y.value_counts())

In [ ]:
# ===========================================
# 4. DIVISIÓN EN CONJUNTOS DE ENTRENAMIENTO
#    Y PRUEBA (SIN FEATURE ENGINEERING)
# ===========================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

In [ ]:
# =======================================
# 5. MODELO BASE k-NN SIN TRANSFORMACIÓN
# =======================================

# Usamos k-NN con k=5 como modelo sencillo de referencia.
# NOTA: aquí no aplicamos ningún tipo de feature engineering
# (no hay escalado, ni nuevas columnas, ni transformaciones).
# Esto es intencional: servirá como línea base para comparar
# después de aplicar feature engineering.

knn_base = KNeighborsClassifier(n_neighbors=5)

# Entrenamiento del modelo con los datos originales
knn_base.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred_base = knn_base.predict(X_test)

# Cálculo de métricas
accuracy_base = accuracy_score(y_test, y_pred_base)

print(f"Accuracy del modelo BASE (sin Feature Engineering): {accuracy_base:.4f}\n")
print("Reporte de clasificación (modelo base):")
print(classification_report(y_test, y_pred_base))

In [ ]:
# ====================================
# 6. MATRIZ DE CONFUSIÓN (MODELO BASE)
# ====================================

cm_base = confusion_matrix(y_test, y_pred_base)

fig, ax = plt.subplots()
im = ax.imshow(cm_base)

ax.set_title("Matriz de confusión - Modelo base (sin FE)")
ax.set_xlabel("Predicción")
ax.set_ylabel("Real")

# Etiquetas de los ejes
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['0', '1'])
ax.set_yticklabels(['0', '1'])

# Mostrar los números en cada celda
for i in range(cm_base.shape[0]):
    for j in range(cm_base.shape[1]):
        ax.text(j, i, cm_base[i, j],
                ha='center', va='center', color='black')

plt.show()

In [ ]:
# =============================================================================
# 7. SECCIÓN PARA FEATURE ENGINEERING (A COMPLETAR POR EL/LA ESTUDIANTE)
# =============================================================================
# En esta sección deberás:
#   - Crear nuevas características a partir de X_train y X_test.
#     Ejemplos:
#        * Transformaciones logarítmicas sobre variables muy sesgadas.
#        * Relaciones entre variables (ratios, productos, diferencias, etc.).
#   - Opcional pero MUY recomendado: aplicar escalado (por ejemplo MinMaxScaler
#     o StandardScaler), recordando:
#        * Ajustar el escalador SOLO con X_train.
#        * Aplicar la transformación al conjunto de prueba X_test.
#
# IMPORTANTE:
#   - NO modifiques X_train y X_test originales.
#   - Trabaja sobre copias: X_train_fe y X_test_fe.
#   - Al final de tus transformaciones, las variables que usará el modelo deben
#     llamarse:
#          X_train_model
#          X_test_model
#
#   De momento, dejamos esta sección de manera que, si no cambias nada,
#   el modelo usará exactamente las mismas características originales.
# =============================================================================

# 7.1 Copiamos los conjuntos originales (NO MODIFICAR ESTAS LÍNEAS)
X_train_fe = X_train.copy()
X_test_fe = X_test.copy()

# 7.2 >>>>> AQUÍ AGREGA TU CÓDIGO DE FEATURE ENGINEERING <<<<<
# Ejemplo de ideas (NO se ejecutan hasta que les quites el comentario):
#
# from sklearn.preprocessing import MinMaxScaler
#
# # a) Crear nuevas columnas (ejemplos hipotéticos):
# X_train_fe['sulfur_ratio'] = X_train_fe['free sulfur dioxide'] / X_train_fe['total sulfur dioxide']
# X_test_fe['sulfur_ratio'] = X_test_fe['free sulfur dioxide'] / X_test_fe['total sulfur dioxide']
#
# # b) Transformación logarítmica (evitar log(0) sumando 1):
# X_train_fe['log_residual_sugar'] = np.log1p(X_train_fe['residual sugar'])
# X_test_fe['log_residual_sugar'] = np.log1p(X_test_fe['residual sugar'])
#
# # c) Escalado Min-Max:
# scaler = MinMaxScaler()
# X_train_fe = pd.DataFrame(
#     scaler.fit_transform(X_train_fe),
#     columns=X_train_fe.columns,
#     index=X_train_fe.index
# )
# X_test_fe = pd.DataFrame(
#     scaler.transform(X_test_fe),
#     columns=X_test_fe.columns,
#     index=X_test_fe.index
# )
#
# IMPORTANTE: si usas MinMaxScaler u otro escalado, NO olvides importar
# la clase correspondiente en la celda de librerías.

# 7.3 AL FINAL de tus transformaciones, asegúrate de que estas variables
#     apunten a las versiones definitivas que quieres usar en el modelo:

X_train_model = X_train_fe   # <- Reemplaza si usas versión escalada, etc.
X_test_model = X_test_fe     # <- Reemplaza si usas versión escalada, etc.

print("Dimensiones de X_train_model y X_test_model después de Feature Engineering:")
print(X_train_model.shape, X_test_model.shape)

In [ ]:
# =====================================================
# 8. MODELO k-NN DESPUÉS DE FEATURE ENGINEERING
# =====================================================

# Importante: aquí se usa X_train_model y X_test_model,
# que pueden contener nuevas características y/o datos escalados.

knn_fe = KNeighborsClassifier(n_neighbors=5)

knn_fe.fit(X_train_model, y_train)
y_pred_fe = knn_fe.predict(X_test_model)

accuracy_fe = accuracy_score(y_test, y_pred_fe)

print(f"Accuracy del modelo con Feature Engineering: {accuracy_fe:.4f}\n")
print("Reporte de clasificación (con Feature Engineering):")
print(classification_report(y_test, y_pred_fe))

In [ ]:
# =========================================
# 9. COMPARACIÓN Y PREGUNTAS DE REFLEXIÓN
# =========================================

print(f"Accuracy modelo BASE (sin FE):   {accuracy_base:.4f}")
print(f"Accuracy modelo con FE:          {accuracy_fe:.4f}")

mejora = accuracy_fe - accuracy_base
print(f"\nMejora absoluta en accuracy: {mejora:.4f}")

print("""
Reflexiona y responde (puedes hacerlo en una celda de Markdown aparte):

1. ¿Qué transformaciones de Feature Engineering aplicaste?
2. ¿Qué variables nuevas creaste y por qué?
3. ¿Cómo cambiaron las métricas (accuracy, precision, recall, F1)?
4. ¿Por qué crees que esas transformaciones ayudaron (o no ayudaron) al modelo?
""")